# Face-Recognition
Este notebook tem como objetivo fornecer um meio de avaliar os modelos presentes no Face-Recognition utilizando o dataset LFW.

## Importações e Inicialização

In [ ]:
import sys
import os
import torch
import numpy as np

sys.path.append('../../src/models/face-recognition')
from evaluate import eval, extract_deep_features, eval_accuracy, find_best_threshold, calculate_tar_far_frr
from models import (
    sphere20,
    sphere36,
    sphere64,
    MobileNetV1,
    MobileNetV2,
    mobilenet_v3_small,
    mobilenet_v3_large
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")import sys
import os
import torch
import numpy as np

sys.path.append('../../src/models/face-recognition')
from evaluate import eval, extract_deep_features, eval_accuracy, find_best_threshold, calculate_tar_far_frr
from models import (
    sphere20,
    sphere36,
    sphere64,
    MobileNetV1,
    MobileNetV2,
    mobilenet_v3_small,
    mobilenet_v3_large
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Selecionar Modelo e Checkpoint

In [2]:
# Modelos Disponíveis
available_models = {
    "sphere20": sphere20,
    "sphere36": sphere36,
    "sphere64": sphere64,
    "mobilenetv1": MobileNetV1,
    "mobilenetv2": MobileNetV2,
    "mobilenetv3_small": mobilenet_v3_small,
    "mobilenetv3_large": mobilenet_v3_large
}

print("Available models:")
for i, model_name in enumerate(available_models.keys(), 1):
    print(f"{i}. {model_name}")

# Selecionar peso
model_name = "mobilenetv3_large"
checkpoint_name = "mobilenetv3_large_5"
embedding_dim = 512
model_path = f"../../src/models/face-recognition/weights/{checkpoint_name}.ckpt"

print(f"\nSelected model: {model_name}")
print(f"Model checkpoint: {model_path}")

Available models:
1. sphere20
2. sphere36
3. sphere64
4. mobilenetv1
5. mobilenetv2
6. mobilenetv3_small
7. mobilenetv3_large

Selected model: mobilenetv3_large
Model checkpoint: ../../src/models/face-recognition/weights/mobilenetv3_large_5.ckpt


## Carregar Modelo

In [3]:
model_class = available_models[model_name]
model = model_class(embedding_dim=embedding_dim)

if model_path and os.path.exists(model_path):
    # Carregar checkpoint completo
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    # Verificar se é um checkpoint completo ou apenas pesos
    if 'model' in checkpoint:
        # É um checkpoint completo - extrair apenas o modelo
        model_state_dict = checkpoint['model']
        print(f"Loading complete checkpoint from epoch {checkpoint.get('epoch', 'unknown')}")
    else:
        # São apenas os pesos do modelo
        model_state_dict = checkpoint
        print("Loading model weights only")
    
    model.load_state_dict(model_state_dict)
    print(f"Model loaded from: {model_path}")
else:
    print(f"Warning: Checkpoint not found at {model_path}")
    print("Evaluating with random weights...")

model = model.to(device)
model.eval()
print("Model ready for evaluation")

Loading complete checkpoint from epoch 4
Model loaded from: ../../src/models/face-recognition/weights/mobilenetv3_large_5.ckpt
Model ready for evaluation


## Configurar Caminho do Dataset LFW

In [4]:
lfw_dataset_path = "../../data/raw/lfw"  # Adjuste conforme necessário

print(f"LFW dataset path: {lfw_dataset_path}")
if os.path.exists(lfw_dataset_path):
    print("✓ Path exists")
else:
    print("✗ Warning: Path does not exist!")

LFW dataset path: ../../data/raw/lfw
✓ Path exists


## Cálculo das Métricas no LFW

In [ ]:
print("Starting evaluation on LFW dataset...")
print("="*50)

# Executar avaliação com as novas métricas
similarity_score, predictions, metrics = eval(
    model, 
    model_path=None, 
    lfw_root=lfw_dataset_path, 
    device=device
)

# Calcular acurácia no notebook
if len(predictions) > 0:
    accuracy = eval_accuracy(predictions, 0.35)
    
    print("="*50)
    print(f"\nEvaluation complete!")
    print(f"Average Similarity Score: {similarity_score:.4f}")
    print(f"LFW Accuracy: {accuracy:.4f}")
    print(f"\nBiometric Metrics at threshold 0.35:")
    print(f"TAR (True Acceptance Rate): {metrics['TAR']:.4f}")
    print(f"FAR (False Acceptance Rate): {metrics['FAR']:.4f}")
    print(f"FRR (False Rejection Rate): {metrics['FRR']:.4f}")
    print(f"\nTotal pairs evaluated: {len(predictions)}")
else:
    print("="*50)
    print(f"\nEvaluation complete!")
    print(f"Average Similarity Score: {similarity_score:.4f}")
    print(f"No valid pairs found for accuracy calculation")
    print(f"Total pairs evaluated: 0")

Starting evaluation on LFW dataset...
LFW - Avaliacao Simplificada (Somente Pares Positivos):
Similaridade Media: 0.6256 | Desvio Padrao: 0.1339

Evaluation complete!
Average Similarity Score: 0.6256
LFW Accuracy: 0.9650
Total pairs evaluated: 3000


## Análise em Diferentes Tresholds

In [ ]:
if len(predictions) > 0:
    print("\n" + "="*60)
    print("ANÁLISE DE MÉTRICAS EM DIFERENTES THRESHOLDS")
    print("="*60)
    
    # Testar diferentes thresholds
    test_thresholds = [0.25, 0.30, 0.35, 0.40, 0.45, 0.50]
    
    print(f"\n{'Threshold':<12} {'TAR':<10} {'FAR':<10} {'FRR':<10} {'Accuracy':<10}")
    print("-" * 60)
    
    for thresh in test_thresholds:
        metrics_at_thresh = calculate_tar_far_frr(predictions, thresh)
        acc_at_thresh = eval_accuracy(predictions, thresh)
        
        print(f"{thresh:<12.2f} {metrics_at_thresh['TAR']:<10.4f} "
              f"{metrics_at_thresh['FAR']:<10.4f} {metrics_at_thresh['FRR']:<10.4f} "
              f"{acc_at_thresh:<10.4f}")
    
    # Encontrar o melhor threshold (EER - Equal Error Rate)
    print("\n" + "="*60)
    print("MELHOR THRESHOLD (Minimizando EER)")
    print("="*60)
    
    best_threshold, best_metrics = find_best_threshold(predictions)
    best_accuracy = eval_accuracy(predictions, best_threshold)
    
    print(f"\nBest Threshold: {best_threshold:.4f}")
    print(f"TAR: {best_metrics['TAR']:.4f}")
    print(f"FAR: {best_metrics['FAR']:.4f}")
    print(f"FRR: {best_metrics['FRR']:.4f}")
    print(f"Accuracy: {best_accuracy:.4f}")
    print(f"EER (|FAR - FRR|): {abs(best_metrics['FAR'] - best_metrics['FRR']):.4f}")
else:
    print("No predictions to analyze.")

## Análise do Resultado

In [6]:
if len(predictions) > 0:
    similarities = predictions[:, 2].astype(float)
    
    # Estatísticas
    print("Similarity Statistics:")
    print(f"Mean: {np.mean(similarities):.4f}")
    print(f"Std: {np.std(similarities):.4f}")
    print(f"Min: {np.min(similarities):.4f}")
    print(f"Max: {np.max(similarities):.4f}")
    print(f"Median: {np.median(similarities):.4f}")
else:
    print("No predictions to analyze.")

Similarity Statistics:
Mean: 0.6256
Std: 0.1339
Min: 0.1134
Max: 0.9901
Median: 0.6407


## Avaliar Vários Modelos (Batch Evaluation)

In [ ]:
models_to_evaluate = [
    ("sphere20", "../../src/models/face-recognition/weights/sphere20_mcp.ckpt"),
    ("mobilenetv1", "../../src/models/face-recognition/weights/mobilenetv1_mcp.ckpt"),
    ("mobilenetv2", "../../src/models/face-recognition/weights/mobilenetv2_mcp.ckpt"),
    # Adicione mais modelos conforme necessário
]

results = []
print("Batch Evaluation")
print("="*50)

for model_name, checkpoint_path in models_to_evaluate:
    if not os.path.exists(checkpoint_path):
        print(f"Skipping {model_name}: checkpoint not found at {checkpoint_path}")
        continue
    
    print(f"\nEvaluating {model_name}...")
    
    # Inicializar modelo
    model_class = available_models[model_name]
    model = model_class(embedding_dim=512).to(device)
    
    # CORREÇÃO: Passar o checkpoint_path para carregar o modelo treinado
    score, preds = eval(model, model_path=checkpoint_path, lfw_root=lfw_dataset_path, device=device)
    
    results.append({
        'model': model_name,
        'score': score,
        'num_pairs': len(preds)
    })

print("\n" + "="*50)
print("EVALUATION SUMMARY")
print("="*50)

for result in results:
    print(f"{result['model']:20s} | Score: {result['score']:.4f} | Pairs: {result['num_pairs']}")

## Inspeção de Predição de Samples

In [ ]:
if len(predictions) > 0:
    print("Sample Predictions (first 5):")
    print("="*50)
    
    for i in range(min(5, len(predictions))):
        path1, path2, similarity, gt = predictions[i]
        print(f"\nPair {i+1}:")
        print(f"  Image 1: {path1}")
        print(f"  Image 2: {path2}")
        print(f"  Similarity: {float(similarity):.4f}")
        print(f"  Ground Truth: {'Same' if gt == '1' else 'Different'}")
else:
    print("No predictions to display.")